# CT padding pipeline
Use sinogram padding to mitigate ROI (circle/cupping) artifacts.

This notebook provides a pipeline for reconstruction using the padding method to mitigate the ROI problem, which can lead to circle and cupping artifacts. An ordinary reconstruction is first done to see whether such artifacts are present and for comparison later in the notebook. There is also export functionality at the end.

### **Python module imports**

In [ ]:
# General imports
import glob, os, pathlib
import psutil
import qim3d
import cil
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from ipywidgets import interact, interactive, IntSlider
import math
import h5py

In [ ]:
# CIL imports
from cil.io import ZEISSDataReader, NikonDataReader
from cil.processors import TransmissionAbsorptionConverter
from cil.processors import Slicer
from cil.recon import FDK
from cil.utilities.jupyter import islicer
from cil.utilities.display import show_geometry, show2D

In [ ]:
# Visualization utility function
def comparator_widget(arr1, arr2, titles=None, cmap='grey', vrange=None):
    axis_slider = IntSlider(min=0, max=len(arr1.shape) - 1, step=1, value=0, description="Axis")
    index_slider = IntSlider(min=0, max=arr1.shape[0] - 1, step=1, value=arr1.shape[0] // 2, description="Index")
    
    if titles is None:
        titles = ["", ""]
    
    if vrange is None:
        vmin = (arr1.min(), arr2.min())
        vmax = (arr1.max(), arr2.max())
    else:
        vmin = (vrange[0], vrange[0])
        vmax = (vrange[1], vrange[1])

    # Function to update and display the comparison
    def array_comparator(axis=0, index=0):
        # Update the index slider range dynamically based on the selected axis
        index_slider.max = arr1.shape[axis] - 1
        if index > index_slider.max:
            index_slider.value = index_slider.max
            return

        slice1 = np.take(arr1, index_slider.value, axis=axis)
        slice2 = np.take(arr2, index_slider.value, axis=axis)

        fig, ax = plt.subplots(1, 2, figsize=(12, 6))

        im = ax[0].imshow(slice1, cmap=cmap, vmin=vmin[0], vmax=vmax[0])
        pos = ax[0].get_position()
        fig.colorbar(im, ax=ax[0], location='left', fraction=0.046, pad=0.15)
        ax[0].set_title(titles[0])

        im = ax[1].imshow(slice2, cmap=cmap, vmin=vmin[1], vmax=vmax[1])
        fig.colorbar(im, ax=ax[1], location='right', fraction=0.046, pad=0.15)
        ax[1].set_title(titles[1])

        # fig.tight_layout()
        plt.show()

    return interactive(array_comparator, axis=axis_slider, index=index_slider)

### **User parameters**

All the parameters required for the pipeline are set here in the beginning of the notebook. They are also mentioned where they are used. Here they are described in detail:

- `ct_path`: the path to the CT reconstruction metadata file. Should be from **Nikon** (`.xtekct`) or **ZEISS** (`.txrm`).

- `center_height`: determines how many center-most slices to use from the projections. Setting it to `'full'` uses the whole projection volume. `center_height` cannot exceed the height of the projections.

- `pad_factor`: the amount of padding to add at each side of the sinogram in the padding method. Thus setting `pad_factor = 0.25` will make the sinogram 50% bigger. From previous experiments `pad_factor` should be set to at least `0.25` to make the method work properly (pushing the artifacts outside of the ROI). There might be benefits from choosing an even higher `pad_factor`, but this is a case-by-case basis and depends on the specific dataset.

- `save_to_disk`: boolean variable for toggling saving the padded reconstruction. **If `save_to_disk` is set to `True`, then an existing file may be overwritten.**

- `clip_range`: the intensity range that will be used to clip the volume when exporting.

- `base_filename`: the base filename will be used for exporting. The file is saved under the HDF5 format and the extension `.h5` will be appended.

In [ ]:
ct_path = '/dtu/3d-imaging-center/projects/2021_DANFIX_Casper/raw_data_3DIM/Casper_top_3_2 [2021-03-17 16.54.39]/Casper_top_3_2_recon.xtekct'
center_height = 100
pad_factor = 0.25
save_to_disk = True
clip_range = (0.0, 0.07)
base_filename = 'top_3_2_padded'

### **Data reading and processing**

For faster processing, we may work on a subset of slices. `center_height` determines how many of the center-most slices should be used. Set to `'full'` to use full volume.

In [ ]:
def create_reader(file_name, roi=None):
    if file_name.endswith('txrm'):
        DataReader = ZEISSDataReader
    elif file_name.endswith('xtekct'):
        DataReader = NikonDataReader
    else:
        raise ValueError("Unrecognizable CT metadata file. File extension should either be '.txrm' or '.xtekct'")
    
    if roi is None:
        return DataReader(file_name=file_name)
    else:
        return DataReader(file_name=file_name, roi=roi)

def get_pixel_nums(ct_path):
    reader = create_reader(file_name=ct_path)
    num_pixels_h = reader.get_geometry().pixel_num_h
    num_pixels_v = reader.get_geometry().pixel_num_v
    return num_pixels_h, num_pixels_v

num_pixels_h, num_pixels_v = get_pixel_nums(ct_path)

if center_height == 'full':
    reader = create_reader(file_name=ct_path)
else:
    slice_dict = {'vertical': (
        num_pixels_v // 2 - center_height // 2,
        num_pixels_v // 2 + center_height // 2,
        1
    )}
    reader = create_reader(file_name=ct_path, roi=slice_dict)

Reading the data might take some time.

In [ ]:
data = reader.read()

Usually the data is given in the transmission domain and thus we convert to the absorption domain.

In [ ]:
data = TransmissionAbsorptionConverter(accelerated=False)(data)

### **Ordinary reconstruction**

In [ ]:
data.reorder(order='tigre')
recon = FDK(data).run(verbose=0)

Here we inspect the reconstruction for artifacts to see if padding is necessary.

In [ ]:
# islicer(recon)
qim3d.viz.line_profile(recon.as_array())

### **Reconstruction with padding**

In [ ]:
from cil.processors import Padder

dim_horizontal = data.get_data_axes_order().index('horizontal')
pad_width = round(pad_factor * data.shape[dim_horizontal])
data_padded = Padder.edge(pad_width={'horizontal': pad_width})(data)

In [ ]:
data.shape, data_padded.shape

In the following, the padded data is passed as a sinogram to FDK, while the original image geometry is used for the volume to reconstruct. This is more efficient than the default behaviour of reconstructing on a correspondingly padded volume that we would have to crop to the original size anyway.

In [ ]:
data_padded.reorder(order='tigre')
ig = data.geometry.get_ImageGeometry()

In [ ]:
del data # this saves a good amount of memory

In [ ]:
recon_padded = FDK(data_padded, ig).run(verbose=0)

In [ ]:
print(recon_padded)

In [ ]:
# islicer(recon_padded, axis_labels=list(recon.dimension_labels))
qim3d.viz.line_profile(recon_padded.as_array())

### **Visual comparison**

The parameter `vrange` specifies the ranges to use. Not specifying it or setting it to `None` uses each of the volumes full intensity range. Experimenting with `vrange` can be used to finetune `clip_range`.

In [ ]:
vrange = clip_range
comparator_widget(recon.as_array(), recon_padded.as_array(), titles=['No padding', 'Padding'], vrange=vrange)

### **Exporting**

Choose the interval `clip_range` to clip the data to and the `base_filename` to save to (the extension is added automatically). The `clip_range` and `pad_factor` will be stored as metadata. The reconstruction volume will be converted to uint16 for exporting.

In [ ]:
h5_filename = f'{base_filename}.h5'

In [ ]:
if save_to_disk:
    vol = np.clip(recon_padded.as_array(), a_min=clip_range[0], a_max=clip_range[1])
    vol = (vol - clip_range[0]) / (clip_range[1] - clip_range[0])
    vol = (vol * (2**16 - 1)).astype(np.uint16)
    with h5py.File(h5_filename, 'w') as f:
        f.create_dataset('recon_vol', data=vol)
        f.attrs['clip_range'] = clip_range
        f.attrs['pad_factor'] = pad_factor

Below we show how to read the exported file back into memory.

In [ ]:
with h5py.File(h5_filename, 'r') as f:
    vol = f['recon_vol'][...]

In [ ]:
qim3d.viz.line_profile(vol)